## Define Constants

In [1]:
import numpy as np

n_queens = 8
n_generations = 1000
population_size = 100
mutation_rate = 0.05
crossover_rate = 0.7

np.random.seed(23)

## Define Auxiliary Functions

In [2]:
def create_initial_population():
    """
    Start with a random population
    """
    return np.random.randint(0, high=n_queens, size=(population_size, n_queens), dtype=int)

In [3]:
def row_score(individual):
    """
    Counts the number of queens in the same row 
    """
    score = 0
    rows_dict = {}
    for row in individual:
        rows_dict[row] = rows_dict.get(row, 0) + 1
        score += 1
        
    return score - len(rows_dict)

In [4]:
def count_diagonal(row, col, row_speed, col_speed, individual):
    """
    Auxiliary function to count the number of queens in a given direction
    """
    count = 0
    row += row_speed
    col += col_speed
    while (row >= 0 and row < n_queens and col >= 0 and col < n_queens):
        if individual[col] == row:
            count += 1
        
        row += row_speed
        col += col_speed
    
    return count

def diagonal_score(individual):
    """
    Count the number of queens in the four diagonals
    """
    score = 0
    for col in range (n_queens):
        row = individual[col]
        score += count_diagonal(row, col, -1, -1, individual)
        score += count_diagonal(row, col, -1, 1, individual)
        score += count_diagonal(row, col, 1, 1, individual)
        score += count_diagonal(row, col, 1, -1, individual)
    
    return score

In [5]:
def fitness(individual):
    """
    The score of an individual is the number of queens in the same row and in the four diagonals
    """
    return row_score(individual) + diagonal_score(individual)

In [6]:
def tournament_selection(pop):
    """
    Select two individuals randomly and survives the best of the two. 
    """
    new_pop = np.zeros(pop.shape, dtype=int)
    for i in range(population_size):
        ind_a = np.random.randint(0, high=population_size, dtype=int)
        ind_b = np.random.randint(0, high=population_size, dtype=int)
        
        if (fitness(pop[ind_a, :]) < fitness(pop[ind_b, :])):
            new_pop[i, :] = pop[ind_a, :]
        else:
            new_pop[i, :] = pop[ind_b, :]
    
    return new_pop

In [7]:
def crossover(pop):
    """
    Executes the crossover operation. With a certain probability two random indviduals create offspring.
    Otherwise, both individuals pass to the next generation.
    """
    new_pop = np.zeros(pop.shape, dtype=int)
    for i in range(population_size // 2):
        ind_a = np.random.randint(0, high=population_size, dtype=int)
        ind_b = np.random.randint(0, high=population_size, dtype=int)
        
        if np.random.uniform(low=0.0, high=1.0) < crossover_rate:
            crossover_point = np.random.randint(1, high=n_queens, dtype=int)
            new_pop[2*i, :crossover_point] = pop[ind_a, :crossover_point]
            new_pop[2*i, crossover_point:] = pop[ind_b, crossover_point:]
            
            new_pop[2*i + 1, :crossover_point] = pop[ind_b, :crossover_point]
            new_pop[2*i + 1, crossover_point:] = pop[ind_a, crossover_point:]
        else:
            new_pop[2*i, :] = pop[ind_a, :]
            new_pop[2*i + 1, :] = pop[ind_b, :]
    
    return new_pop
    

In [8]:
def mutate_row(row):
    """
    Changes a specific value of an individual with a random value according to the mutation probability.
    """
    if np.random.uniform(low=0.0, high=1.0) < mutation_rate:
        return np.random.randint(0, high=n_queens, dtype=int)
    return row


def mutation(pop):
    """
    Executes the mutation operation with the whole population
    """
    new_pop = np.zeros(pop.shape, dtype=int)
    for i in range(population_size):
        new_pop[i, :] = np.array([mutate_row(row) for row in pop[i, :]])
        
    return new_pop

## Testing the Functions

In [9]:
population = create_initial_population()
print(population.shape)
print(population[:5, :])

(100, 8)
[[3 6 0 1 6 0 7 5]
 [4 3 7 2 1 3 6 3]
 [5 4 1 2 3 5 5 0]
 [4 5 0 1 7 2 6 1]
 [3 1 3 7 7 4 1 1]]


In [10]:
fitness(population[0, :])

8

In [11]:
selected_population = tournament_selection(population)
selected_population[:5, :]

array([[7, 1, 7, 3, 4, 0, 5, 5],
       [5, 2, 1, 6, 1, 7, 1, 2],
       [3, 1, 3, 7, 7, 4, 1, 1],
       [5, 2, 5, 2, 4, 4, 0, 6],
       [0, 6, 1, 1, 6, 1, 5, 6]])

In [12]:
crossover_population = crossover(selected_population)
crossover_population[:5, :]

array([[1, 2, 6, 6, 7, 3, 4, 2],
       [2, 1, 6, 6, 1, 0, 7, 4],
       [0, 7, 0, 0, 6, 1, 6, 1],
       [2, 1, 6, 4, 0, 7, 1, 4],
       [2, 1, 1, 6, 1, 0, 7, 4]])

In [13]:
mutation_population = mutation(crossover_population)
mutation_population[:5, :]

array([[1, 2, 6, 6, 7, 3, 4, 2],
       [2, 1, 6, 6, 1, 0, 7, 4],
       [0, 7, 0, 5, 6, 1, 6, 1],
       [2, 1, 6, 4, 0, 7, 1, 4],
       [2, 1, 1, 6, 1, 0, 7, 4]])

## Run the Genetic Algorithm

1. Create an initial population
1. Selection of the new population (The stronger wins)
1. Crossover Operation (Generates Offspring)
1. Mutation Operation (Changes certain values with a small probability)
1. Move to step 2 until convergence.

In [14]:
## RUN The Genetic Algorithm

population = create_initial_population()

for gen in range(n_generations):
    population = tournament_selection(population)
    population = crossover(population)
    population = mutation(population)
    
    best_individual = min(population, key=lambda x: fitness(x))
    print(gen + 1,  ": best fitness = ", fitness(best_individual))
    
    if (fitness(best_individual) == 0):
        break

print("\nbest individual = ", best_individual)
print("best fitness: ", fitness(best_individual))

1 : best fitness =  3
2 : best fitness =  4
3 : best fitness =  4
4 : best fitness =  4
5 : best fitness =  3
6 : best fitness =  3
7 : best fitness =  3
8 : best fitness =  2
9 : best fitness =  1
10 : best fitness =  2
11 : best fitness =  2
12 : best fitness =  1
13 : best fitness =  1
14 : best fitness =  1
15 : best fitness =  1
16 : best fitness =  1
17 : best fitness =  1
18 : best fitness =  1
19 : best fitness =  1
20 : best fitness =  1
21 : best fitness =  1
22 : best fitness =  1
23 : best fitness =  1
24 : best fitness =  1
25 : best fitness =  1
26 : best fitness =  1
27 : best fitness =  1
28 : best fitness =  1
29 : best fitness =  1
30 : best fitness =  1
31 : best fitness =  1
32 : best fitness =  1
33 : best fitness =  0

best individual =  [3 1 4 7 5 0 2 6]
best fitness:  0
